In [1]:
import pandas as pd

#### Nuotekos

In [6]:
nuotekos = pd.read_csv("csv/nuotekos.csv")
nuotekos = nuotekos.replace("Klaipėdos m. sav.", "Klaipėda").replace("Kauno m. sav.", "Kaunas").replace("Vilniaus m. sav.", "Vilnius").replace("Panevėžio m. sav.", "Panevėžys").replace("Šiaulių m. sav.", "Šiauliai").replace("Alytaus m. sav.", "Alytus")
nuotekos = nuotekos.drop(['Rodiklis', "Matavimo vienetai"], 1)
nuotekos = pd.merge(
    nuotekos[nuotekos["Išvalymas"] == "Išleista išvalytų iki normos nuotekų"],
    nuotekos[nuotekos["Išvalymas"] == "Iš viso išleista nuotekų"],
    on=["Administracinė teritorija", "Laikotarpis"])
nuotekos["y"] = round(100*nuotekos["Reikšmė_x"]/nuotekos["Reikšmė_y"],2)
nuotekos = nuotekos.drop(['Išvalymas_x', "Išvalymas_y", "Reikšmė_x", "Reikšmė_y"], 1)
nuotekos = nuotekos.rename({'Laikotarpis': 'x', 'Administracinė teritorija': 'city'}, axis=1)
nuotekos.to_csv("kpi/nuotekos.csv", index = False)

#### Metalo laužas

In [7]:
mlauzas = pd.read_csv("csv/metalo_lauzas.csv")
mlauzas = mlauzas[(mlauzas["Supirkimo šaltinis"] == "Supirkta, iš viso")]
mlauzas = mlauzas.drop(['Rodiklis', "Matavimo vienetai", "Supirkimo šaltinis"], 1)
mlauzas = pd.merge(
    mlauzas[mlauzas["Metalų laužo rūšis"] == "Juodųjų metalų laužas, tūkst. tonų"],
    mlauzas[mlauzas["Metalų laužo rūšis"] == "Spalvotųjų metalų laužas, tonos"],
    on=["Apskritys", "Laikotarpis"])
mlauzas["y"] = mlauzas["Reikšmė_x"] + mlauzas["Reikšmė_y"]
mlauzas = mlauzas.drop(['Metalų laužo rūšis_x', "Metalų laužo rūšis_y", "Reikšmė_x", "Reikšmė_y"], 1)
mlauzas = mlauzas.rename({'Laikotarpis': 'x', 'Apskritys': 'city'}, axis=1)
mlauzas.to_csv("kpi/mlauzas.csv", index = False)